In [33]:
%load_ext autoreload
%autoreload 2
from concat_configuration import ConcatConfiguration
from count_configuration import CountConfiguration
from cached_runner import CachedRunner
from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
%%capture
count_configuration = CountConfiguration()
count_runner = CachedRunner(count_configuration)
reduced_matrixes = count_runner.get_datasets(fresh=False)

In [35]:
reduced_all = reduced_matrixes[0]
reduced_group_year = reduced_matrixes[1]
reduced_year_channels = reduced_matrixes[2]
reduced_flat = reduced_matrixes[3]
reduced_group_category = reduced_matrixes[4]

In [36]:
%%capture
reduced_matrixes_with_context = count_runner.get_datasets(fresh=False, filter_function=count_configuration.all_words_have_context)

In [37]:
context_reduced_all = reduced_matrixes_with_context[0]
context_reduced_group_year = reduced_matrixes_with_context[1]
context_reduced_year_channels = reduced_matrixes_with_context[2]
context_reduced_flat = reduced_matrixes_with_context[3]
context_reduced_group_category = reduced_matrixes_with_context[4]

In [38]:
display(reduced_all)

,group,group_members,total_words,in_a_row,count_0,repetitions_0,count_1,repetitions_1,count_2,repetitions_2,count_3,repetitions_3,count_4,repetitions_4,count_5,repetitions_5,count_6,repetitions_6,count_7,repetitions_7,count_8,repetitions_8,count_9,repetitions_9,valence_only_negative,valence_mostly_negative,valence_only_neutral,valence_mostly_neutral,valence_only_mixed,valence_mostly_mixed,valence_only_positive,valence_mostly_positive
0,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,1,olema,123829,mina,68988,ei,53058,saama,31116,siis,27272,ka,24635,tegema,21751,kui,21391,pidama,17334,@,15786,1459275,27146,670888,35777,0,112789,529350,18138
1,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,2,mina olema,9874,mina ei,8305,ei olema,8233,ei saama,6746,ei teadma,4012,olema ka,3180,siis olema,2752,@ ago,2451,@ kadri,2333,kui mina,1983,1459275,27146,670888,35777,0,112789,529350,18138
2,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,3,mina ei teadma,1765,mina ei saama,1489,@ sebastian sõeruer,1448,ei saama aru,927,mina ei olema,889,mina olema ka,414,mina ei tahtma,367,mina ei oskama,347,mina saama aru,280,ei teadma kas,252,1459275,27146,670888,35777,0,112789,529350,18138
3,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,4,mina ei saama aru,419,🌱 🌱 🌱 🌱,113,mina ei teadma kas,100,░ ░ ░ ░,98,ei saama aru miks,79,tere hommikune @ piret,71,@ piret @ piret,68,piret @ piret @,57,pizza with a price,57,@ triinuadelaide | triinu.viikholm,54,1459275,27146,670888,35777,0,112789,529350,18138


In [39]:
display(context_reduced_all)

,group,group_members,total_words,in_a_row,count_0,repetitions_0,count_1,repetitions_1,count_2,repetitions_2,count_3,repetitions_3,count_4,repetitions_4,count_5,repetitions_5,count_6,repetitions_6,count_7,repetitions_7,count_8,repetitions_8,count_9,repetitions_9,valence_only_negative,valence_mostly_negative,valence_only_neutral,valence_mostly_neutral,valence_only_mixed,valence_mostly_mixed,valence_only_positive,valence_mostly_positive
0,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,1,mina,68988,siis,27272,kui,21391,kas,12701,jah,10935,nii,10888,ära,10699,ikka,8626,hea,7654,vist,7315,1459275,27146,670888,35777,0,112789,529350,18138
1,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,2,kui mina,1983,sebastian sõeruer,1460,siis mina,1122,kas mina,1052,tere hommikune,917,väga hea,679,nii palju,676,kuidas mina,638,miks mina,627,siis kui,532,1459275,27146,670888,35777,0,112789,529350,18138
2,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,3,nagu mina aru,55,siis kui mina,47,list index out,46,kui mina nüüd,45,nii palju kui,39,the file name,39,check the file,38,check the modu,37,check the function,37,kui mina õigesti,32,1459275,27146,670888,35777,0,112789,529350,18138
3,all,"['EX / ex01-id-code', 'EX / ex02-cpu', 'EX / e...",2876496,4,check the file name,38,inval litera for int,28,line require between summary,17,summary line and description,17,require between summary line,17,blank line require between,17,between summary line and,17,mina kood pilk peale,15,maximum recursion depth exceede,14,testing faile with message,12,1459275,27146,670888,35777,0,112789,529350,18138


In [40]:
from summa.summarizer import summarize

In [41]:
%%capture
concat_configuration = ConcatConfiguration()
concat_runner = CachedRunner(concat_configuration)
reduced_matrixes = concat_runner.get_datasets(fresh=False)